In [181]:
import requests
import pandas as pd
# Import required info for access to the reddit api
import clientInfo as botInfo

In [182]:
base_url = 'https://www.reddit.com/'
data = {'grant_type': 'password', 'username': botInfo.username, 'password': botInfo.password}
auth = requests.auth.HTTPBasicAuth(botInfo.client_id, botInfo.secret)
r = requests.post(base_url + 'api/v1/access_token',
                  data=data,
                  headers={'user-agent': 'CommentBot by u/ScraperBot-0'},
		          auth=auth)

d = r.json()

In [183]:
token = 'bearer ' + d['access_token']

base_url = 'https://oauth.reddit.com'

headers = {'Authorization': token, 'User-Agent': 'CommentBot by ScraperBot-0'}
response = requests.get(base_url + '/api/v1/me', headers=headers)

if response.status_code == 200:
    print("Connected")

Connected


In [184]:
subreddit = '/r/Art'
payload = {'limit': 25}
response = requests.get(base_url + subreddit, headers=headers, params=payload).json()

In [185]:
response

{'kind': 'Listing',
 'data': {'after': 't3_wucvy0',
  'dist': 26,
  'modhash': None,
  'geo_filter': None,
  'children': [{'kind': 't3',
    'data': {'approved_at_utc': None,
     'subreddit': 'Art',
     'selftext': "General Discussion threads are for casual chat; a place to ask for recommendations, lists, or creative feedback; to talk about materials, history, or techniques; and anything else that comes to mind.\n\nIf you're looking for information about a particular work of art, /r/WhatIsThisPainting is still the best resource.  /r/drawing , /r/painting , and /r/learnart may also be useful.   /r/ArtistLounge is also a good place for general discussion.  Please see our [list of art-related subs](https://www.reddit.com/r/Art/wiki/related) for more options.\n\n[Rule 8 still applies](https://www.reddit.com/r/Art/wiki/index#wiki_8._be_respectful_and_stay_on_topic) except that questions/complaints about r/Art and Reddit overall are allowed.\n\n***\n[Previous month's discussion](https://ww

In [186]:
pict = requests.get('https://i.redd.it/nzsx5fjai3j91.jpg')

In [187]:
with open('./home', "wb") as f:
    f.write(pict.content)


In [188]:
from PIL import Image
with Image.open('./home') as img:
    width, height = img.size
print("Width: {}\nHeight: {}".format(width, height))

Width: 1920
Height: 3027


In [209]:
import json

posts = response["data"]["children"]
# number of posts
numPosts = len(posts)


def getResolution(datPost):
    info = datPost["preview"]["images"][0]["source"]
    return info["width"], info["height"]

# assign all post
def assignPostData(post: json, postInfo: dict):
    # get the url to the image/gif
    postInfo["url"].append(post["url_overridden_by_dest"])
    # Write flag to show that it is an image or not
    postInfo["image"].append(True if not "gif" in post["url"] else False)
    # Find the width and the height of the image/gif
    postInfo["width"], postInfo["height"] = getResolution(post)
    # Check to see if it's NSFW
    postInfo["NSFW"].append(True if "nsfw" in post["thumbnail"] else False)
    # Get number of upvotes and ratio of upvotes
    postInfo["ups"].append(post["ups"])
    postInfo["upRatio"].append(post["upvote_ratio"])
    

def getPostData(posts):
    # Stores information about the image
    postData = {
        "image":    [],
        "height":   [],
        "width":    [],
        "url":      [],
        'ups':      [],
        "upRatio":  [],
        "NSFW":     []
    }

    for i in range(len(posts)):
        post = posts[i]["data"]
        if "url_overridden_by_dest" in post:
            # assign all data to the post
            assignPostData(post, postData)
    return postData

In [210]:
postData = getPostData(posts)
postData

26


{'image': True,
 'height': 3472,
 'width': 2480,
 'url': 'https://i.redd.it/7desg3ctc5j91.jpg',
 'ups': 37,
 'upRatio': 0.93,
 'NSFW': False}

In [200]:
posts[1]["data"]

{'approved_at_utc': None,
 'subreddit': 'Art',
 'selftext': '',
 'author_fullname': 't2_k50lh',
 'saved': False,
 'mod_reason_title': None,
 'gilded': 0,
 'clicked': False,
 'title': 'aimlessly, me, pixel art, 2022',
 'link_flair_richtext': [{'e': 'text', 't': 'Artwork'}],
 'subreddit_name_prefixed': 'r/Art',
 'hidden': False,
 'pwls': 6,
 'link_flair_css_class': 'artwork',
 'downs': 0,
 'thumbnail_height': 87,
 'top_awarded_type': None,
 'hide_score': False,
 'name': 't3_wu6wpb',
 'quarantine': False,
 'link_flair_text_color': 'dark',
 'upvote_ratio': 0.97,
 'author_flair_background_color': None,
 'ups': 8639,
 'total_awards_received': 12,
 'media_embed': {},
 'thumbnail_width': 140,
 'author_flair_template_id': None,
 'is_original_content': False,
 'user_reports': [],
 'secure_media': None,
 'is_reddit_media_domain': True,
 'is_meta': False,
 'category': None,
 'secure_media_embed': {},
 'link_flair_text': 'Artwork',
 'can_mod_post': False,
 'score': 8639,
 'approved_by': None,
 'is_